# Workload generator

includes :

In [91]:
import json
import gzip
from datetime import datetime

In [92]:
tracesPath='./clusterdata-2011-2'
tasksNums = range(0,1)

tasksFiles=list(map(lambda x:tracesPath+'/task_events/part-'+str(x).zfill(5)+'-of-00500.csv.gz', tasksNums))

print (tasksFiles[0])
print ('to')
print (tasksFiles[-1])

./clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz
to
./clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz


In [93]:
def genJobs(taskfiles):
    tasks={}
    arrival=[]
    current_time=0
    jobs = []
    def task_complete(jID, tID):
        (res, prio, evts) = tasks[(jID, tID)]
        if len(evts) < 3:
            #bad_tasks.append((jID, tID))
            pass
        else:
            subtime=evts[0][1] #timestamp du debut
            runtime=evts[1][1] #event du lancement de la tache
            endtime=evts[-1][1] #fin de la tache
            
            #temps en seconde
            subt=subtime/1000000.0
            #temps d'execution
            walltime=float(endtime-runtime)/1000000.0
            
            job={}
            job['id'] = tID
            job['profile'] = "delay_ft.B.1"
            job['res'] = int(res*100)
            job['subtime'] = subt
            job['walltime'] = walltime
            
            jobs.append(job)
            
            #print(json.dumps(job))
            
            del tasks[(jID, tID)]
    
    
    for f in tasksFiles:
        print ('File: ', f)
        with gzip.open(f, 'rb') as f:
            for line in f.readlines():
                
                """
                1. timestamp
                2. missing info
                3. job ID
                4. task index - within the job
                5. machine ID
                6. event type
                7. user name
                8. scheduling class
                9. priority
                10. resource request for CPU cores
                11. resource request for RAM
                12. resource request for local disk space
                13. different-machine constraint
                """
                
                (timestamp,_,jID,tID,_,evtType,_,_,prio,res,_,_,_)=line.decode("ascii").split(',')
                try:
                    # début de la tache
                    if(evtType == '0'):
                        timestamp=int(timestamp)
                        #on récupère la class, la prio, et le temps depuis le début, qu'on stock
                        tasks[(int(jID), int(tID))] = (float(res), int(prio), [(0,int(timestamp))])
                        #on actualise le temps
                        if timestamp>0:
                            arrival.append(timestamp-current_time)
                            current_time=timestamp
                            
                    # event de la tache en cours        
                    else:
                        
                        timestamp, jID, tID, evtType = int(timestamp), int(jID), int(tID), int(evtType)
                        (_,_, events)=tasks[(jID,tID)]
                        #on rajoute le timestamp de l'evenement
                        events.append((evtType,timestamp))
                        
                        #fin de la tache
                        if evtType == 4 or evtType == 5:
                            task_complete(jID, tID)
                            
                except:
                    pass
                
    return jobs

In [94]:
jsonPath = "./workload.json"

outputData = {}
outputData['description'] ="Workload from google data"
outputData['date'] = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
outputData['jobs'] = genJobs(tasksFiles)

with open(jsonPath, "w") as jsonF:
    jsonF.write(json.dumps(outputData, indent = 4))

File:  ./clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz
